# Task A2 - Identifying the attacker - (Computer Forensics, Packet Analysis)Points: 40

## Problem Statement

Using the timestamp and IP address information from the VPN log, the FBI was able to identify a virtual server that the attacker used for staging their attack. They were able to obtain a warrant to search the server, but key files used in the attack were deleted.

Luckily, the company uses an intrusion detection system which stores packet logs. They were able to find an SSL session going to the staging server, and believe it may have been the attacker transferring over their tools.

The FBI hopes that these tools may provide a clue to the attacker's identity


Downloads:

- Files captured from root's home directory on the staging server ([`root.tar.bz2`](./data/a2/root/))
- PCAP file believed to be of the attacker downloading their tools ([`session.pcap`](./data/a2/session.pcap))

## What to do

What was the username of the account the attacker used when they built their tools?

## My solution

Dependencies:

- [Wireshark](https://www.wireshark.org/)

First, analyze the type of the given file, `session.pcap`.

In [1]:
%%bash
file ./data/a2/session.pcap

./data/a2/session.pcap: pcap capture file, microsecond ts (little-endian) - version 2.4 (Ethernet, capture length 65535)


So, the file is a `pcap` file as the extension shows.

In [1]:
import dpkt
import datetime
from dpkt.utils import mac_to_str, inet_to_str

In [2]:
def print_packets(pcap):
    """Print out information about each packet in a pcap
       Args:
           pcap: dpkt pcap reader object (dpkt.pcap.Reader)
    """
    # For each packet in the pcap process the contents
    for timestamp, buf in pcap:

        # Print out the timestamp in UTC
        print('Timestamp: ', str(datetime.datetime.utcfromtimestamp(timestamp)))

        # Unpack the Ethernet frame (mac src/dst, ethertype)
        eth = dpkt.ethernet.Ethernet(buf)
        print('Ethernet Frame: ', mac_to_str(eth.src), mac_to_str(eth.dst), eth.type)

        # Make sure the Ethernet data contains an IP packet
        if not isinstance(eth.data, dpkt.ip.IP):
            print('Non IP Packet type not supported %s\n' % eth.data.__class__.__name__)
            continue

        # Now access the data within the Ethernet frame (the IP packet)
        # Pulling out src, dst, length, fragment info, TTL, and Protocol
        ip = eth.data

        # Print out the info, including the fragment flags and offset
        print('IP: %s -> %s   (len=%d ttl=%d DF=%d MF=%d offset=%d)\n' %
              (inet_to_str(ip.src), inet_to_str(ip.dst), ip.len, ip.ttl, ip.df, ip.mf, ip.offset))

    # Pretty print the last packet
    print('** Pretty print demo **\n')
    eth.pprint()


def test():
    """Open up a test pcap file and print out the packets"""
    with open('data/session.pcap', 'rb') as f:
        pcap = dpkt.pcap.Reader(f)
        print_packets(pcap)


if __name__ == '__main__':
    test()

FileNotFoundError: [Errno 2] No such file or directory: 'data/session.pcap'